In [1]:
from sklearn import svm
import numpy as np
X = [[0, 0], [1, 1]]
y = [0, 1]
clf = svm.SVC()
clf.fit(np.array(X), np.array(y))  

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [2]:
clf.predict(([[1., 1.]]))

array([1])

In [3]:
%load_ext rpy2.ipython

In [4]:
%%R -o cl_data -o tcga_data -o tcga_hist -o cl_names


load("../data/Endometrial_CN_gistic_bundle.Rdata")

genes <- colnames(GISTIC_cl_endo)
cl_names <- rownames(GISTIC_cl_endo)

y <- rep(0, nrow(TCGA_histology))
y[which(TCGA_histology$CLI_histological_type=="serous endometrial adenocarcinoma")] <- 1

TCGA_histology$y <- as.factor(y)

cl_data <- GISTIC_cl_endo
tcga_data <- GISTIC_tcga_endo
tcga_hist <- TCGA_histology

In [5]:
from random import sample

y=np.array(tcga_hist['y'])
#all_classes=map(int, np.array(tcga_hist['y']))

inds_class1=np.where(y=='1')[0]
inds_class2=np.where(y=='0')[0]

traininds_class1=sample(inds_class1, len(inds_class1)/2)
traininds_class2=sample(inds_class2, len(inds_class1)/2)

testinds_class2=np.setdiff1d(inds_class2, traininds_class2)
testinds_class1=np.setdiff1d(inds_class1, traininds_class1)



In [7]:
type(tcga_data)

numpy.ndarray

In [6]:
y=y.astype(int)
testinds_class1=list(testinds_class1)
testinds_class2=list(testinds_class2)
svm_ucec_cn=svm.SVC(probability=True)
svm_ucec_cn.fit(tcga_data[traininds_class1+traininds_class2,:], y[traininds_class1+traininds_class2])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [7]:
from sklearn.metrics import roc_auc_score
testset_prediction_svm=svm_ucec_cn.predict_proba(tcga_data[testinds_class1+testinds_class2,:])
roc_auc_score(y[testinds_class1+testinds_class2] ,testset_prediction_svm[:,1])

0.92674239507123612

In [8]:
cl_prediction_svm=svm_ucec_cn.predict_proba(cl_data)
cl_names[np.where(cl_prediction_svm[:,0]<0.5)]

array(['MFE280_ENDOMETRIUM', 'KLE_ENDOMETRIUM', 'HEC1A_ENDOMETRIUM',
       'HEC6_ENDOMETRIUM', 'HEC251_ENDOMETRIUM', 'SNGM_ENDOMETRIUM',
       'HEC50B_ENDOMETRIUM', 'SNU1077_ENDOMETRIUM', 'JHUEM3_ENDOMETRIUM',
       'TEN_ENDOMETRIUM'], 
      dtype='|S32')

In [9]:
cl_names

array(['MFE296_ENDOMETRIUM', 'MFE280_ENDOMETRIUM', 'MFE319_ENDOMETRIUM',
       'RL952_ENDOMETRIUM', 'ISHIKAWAHERAKLIO02ER_ENDOMETRIUM',
       'KLE_ENDOMETRIUM', 'EN_ENDOMETRIUM', 'HEC1A_ENDOMETRIUM',
       'JHUEM7_ENDOMETRIUM', 'ESS1_ENDOMETRIUM', 'EFE184_ENDOMETRIUM',
       'JHUEM2_ENDOMETRIUM', 'JHUEM1_ENDOMETRIUM', 'HEC265_ENDOMETRIUM',
       'HEC151_ENDOMETRIUM', 'HEC108_ENDOMETRIUM', 'HEC6_ENDOMETRIUM',
       'HEC59_ENDOMETRIUM', 'HEC251_ENDOMETRIUM', 'SNGM_ENDOMETRIUM',
       'COLO684_ENDOMETRIUM', 'AN3CA_ENDOMETRIUM', 'SNU685_ENDOMETRIUM',
       'HEC50B_ENDOMETRIUM', 'SNU1077_ENDOMETRIUM', 'JHUEM3_ENDOMETRIUM',
       'HEC1B_ENDOMETRIUM', 'TEN_ENDOMETRIUM'], 
      dtype='|S32')